# XML example and exercise
****
+ study examples of accessing nodes in XML tree structure  
+ work on exercise to be completed and submitted
****
+ reference: https://docs.python.org/2.7/library/xml.etree.elementtree.html
+ data source: http://www.dbis.informatik.uni-goettingen.de/Mondial
****

In [75]:
from xml.etree import ElementTree as ET
import pandas as pd

## XML example

+ for details about tree traversal and iterators, see https://docs.python.org/2.7/library/xml.etree.elementtree.html

In [27]:
document = ET.parse( './data/mondial_database_less.xml' )


In [28]:
# print names of all countries
for child in document.getroot():
    print child.find('name').text

Albania
Greece
Macedonia
Serbia
Montenegro
Kosovo
Andorra


In [64]:
# print names of all countries and their cities
for element in document_tree.iterfind('country'):
    print '* ' + element.find('name').text + ':',
    capitals_string = ''
    for subelement in element.getiterator('city'):
        capitals_string += subelement.find('name').text + ', '
    print capitals_string[:-2]

* Albania: Tirana, Shkodër, Durrës, Vlorë, Elbasan, Korçë
* Greece: Komotini, Kavala, Athina, Peiraias, Peristeri, Acharnes, Patra, Kozani, Kerkyra, Ioannina, Thessaloniki, Iraklio, Chania, Ermoupoli, Rhodes, Tripoli, Lamia, Chalkida, Larissa, Volos, Mytilini, Karyes
* Macedonia: Skopje, Kumanovo
* Serbia: Beograd, Novi Sad, Niš
* Montenegro: Podgorica
* Kosovo: Prishtine
* Andorra: Andorra la Vella


****
## XML exercise

Using data in 'data/mondial_database.xml', the examples above, and refering to https://docs.python.org/2.7/library/xml.etree.elementtree.html, find

1. 10 countries with the lowest infant mortality rates
2. 10 cities with the largest population
3. 10 ethnic groups with the largest overall populations (sum of best/latest estimates over all countries)
4. name and country of a) longest river, b) largest lake and c) airport at highest elevation

****
## Quesion 1
* Ten countries with the lowest infant mortality rates
****

In [289]:
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()
cntry = []
infmort = []
for country in root.iter('country'):
    if country.find('name') != None:
        if country.find('infant_mortality') != None:
            cntry.append(country.find('name').text)
            infmort.append(float(country.find('infant_mortality').text))
        else:
            continue
    else:
        continue

data = {'country':cntry, 'infant mortality': infmort}
df = pd.DataFrame(data).sort_values(by='infant mortality').head(10)
df

,country,infant mortality
36,Monaco,1.81
90,Japan,2.13
109,Bermuda,2.48
34,Norway,2.48
98,Singapore,2.53
35,Sweden,2.60
8,Czech Republic,2.63
72,Hong Kong,2.73
73,Macao,3.13
39,Iceland,3.15


****
## Quesion 2
* Ten cities with the largest population
****

In [290]:
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()
cityList = []
popList = []
for city in root.findall('country/city'):
    if city.findtext('name') != None:
        for population in city.findall('population'):
            if population.attrib['year'] != '2011':
                continue;
            else:
                popList.append(int(population.text))
                cityList.append(city.findtext('name'))
data = {'city' : cityList, 'population' : popList}
df = pd.DataFrame(data).sort_values('population', ascending=False).head(10)    
df 
        

,city,population
8,Beograd,1639121
56,Montevideo,1318755
23,Sofia,1270284
39,Yerevan,1060138
42,Kathmandu,1003285
18,Zagreb,686568
52,Kingston,662426
14,Rīga,658640
15,Vilnius,535631
35,Dublin,525383


****
## Quesion 3
* Ten ethnic groups with the largest overall populations
****

In [291]:
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()
ethnicityList = []
popList = []
for country in root.findall('country'):
    for population in reversed(country.findall('population')):
        popList.append(int(population.text))
        for ethnicity in country.findall('ethnicgroup'):
            ethnicityList.append((int(population.text), float(ethnicity.attrib['percentage']), ethnicity.text))
        break

df= pd.DataFrame(ethnicityList, columns=['population', 'percentage', 'ethnicity'])
df['Ethnic Population'] = (df.population * df.percentage)/100
del df['population']
del df['percentage']
df.groupby('ethnicity').sum().sort_values(by='Ethnic Population', ascending=False).head(10)

,Ethnic Population
ethnicity,
Han Chinese,1.245059e+09
Indo-Aryan,8.718156e+08
European,4.948722e+08
African,3.183251e+08
Dravidian,3.027137e+08
Mestizo,1.577344e+08
Bengali,1.467769e+08
Russian,1.318570e+08
Japanese,1.265342e+08


****
## Quesion 4
* Name and country of a) longest river, b) largest lake and c) airport at highest elevation
****


In [292]:
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()
riverList = []
countryList = []
for river in root.findall('river'):
    if (river.find('length') != None and river.find('name') != None):
        if len(river.get('country').split(' ')) == 1:
            riverList.append((float(river.find('length').text), river.find('name').text, river.get('country')))
        else:
            for eachCountry in river.get('country').split(' '):
                riverList.append((float(river.find('length').text), river.find('name').text, eachCountry))

rdf = pd.DataFrame(riverList, columns=['Length', 'Name', 'Code']).sort_values(by='Length', ascending=False)

for country in root.findall('country'):
    countryList.append((country.get('car_code'), country.find('name').text))
cdf = pd.DataFrame(countryList, columns=['Code', 'Country Name'])

rdf = rdf.sort_values(by='Length', ascending=False)
rdf = rdf.merge(cdf, on='Code').sort_values(by='Length', ascending=False)
del rdf['Code']
rdf.head(3)

,Length,Name,Country Name
0,6448.0,Amazonas,Peru
9,6448.0,Amazonas,Colombia
1,6448.0,Amazonas,Brazil


In [293]:
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()

lakeList = []
countryList = []

for lake in root.findall('lake'):
    if (lake.find('area') != None and lake.find('name') != None):
        if len(lake.get('country').split(' ')) == 1:
            lakeList.append((float(lake.find('area').text), lake.find('name').text, lake.get('country')))
        else:
            for eachCountry in lake.get('country').split(' '):
                lakeList.append((float(lake.find('area').text), lake.find('name').text, eachCountry))
                #lakeList.append((float(lake.find('area').text), lake.find('name').text, lake.get('country')))


ldf = pd.DataFrame(lakeList, columns=['Area', 'Name', 'Code']).sort_values(by='Area', ascending=False)
for country in root.findall('country'):
    countryList.append((country.get('car_code'), country.find('name').text))
cdf = pd.DataFrame(countryList, columns=['Code', 'Country Name'])

ldf = ldf.sort_values(by='Area', ascending=False)
ldf = ldf.merge(cdf, on='Code').sort_values(by='Area', ascending=False)
del ldf['Code']
ldf.head(5)

,Area,Name,Country Name
0,386400.0,Caspian Sea,Russia
13,386400.0,Caspian Sea,Turkmenistan
9,386400.0,Caspian Sea,Iran
15,386400.0,Caspian Sea,Kazakhstan
14,386400.0,Caspian Sea,Azerbaijan


In [294]:
document = ET.parse( './data/mondial_database.xml' )
root = document.getroot()
airportList = []
countryList = []
for airport in root.findall('airport'):
    if (airport.find('elevation').text != None and airport.find('name') != None):
        airportList.append((int(airport.find('elevation').text), airport.find('name').text, airport.get('country')))
adf = pd.DataFrame(airportList, columns=['Elevation', 'Name', 'Code']).sort_values(by='Elevation', ascending=False).head(1)

for country in root.findall('country'):
    countryList.append((country.get('car_code'), country.find('name').text))
cdf = pd.DataFrame(countryList, columns=['Code', 'Country Name'])
adf = adf.merge(cdf, on='Code')
del adf['Code']
adf

,Elevation,Name,Country Name
0,4063,El Alto Intl,Bolivia
